In [2]:
import fitz
import os

In [3]:
# import cv2
# from PIL import Image, ImageDraw

# for pdf_path in pdfs:
#     with fitz.open(pdf_path) as pdf_file:
#         for page_index, page in enumerate(pdf_file):
#             # print(page.get_text("blocks"))

#             page_img = page.get_pixmap()
#             page_img = Image.frombytes("RGB", [page_img.width, page_img.height], page_img.samples)
#             synth_img = Image.new("RGB", [page_img.width, page_img.height], "white")
#             draw = ImageDraw.Draw(page_img)
#             draw_synth = ImageDraw.Draw(synth_img)
#             html = page.get_text("html")
#             with open(f"{pdf_path}_{page_index}.html", "w") as f:
#                 f.write(html)
#             tabs = page.find_tables()
#             if tabs.tables:
#                 for table in tabs.tables:
#                     page.add_redact_annot(table.bbox)
#                     bbox = table.bbox
#                     cells = table.cells
#                     cell_contents = table.extract()
#                     draw.rectangle(bbox, outline="green")
#                     draw_synth.rectangle(bbox, outline="green")
#                     draw_synth.text(
#                         (bbox[2], bbox[3]), f"{len(cells)}", fill="black", font_size=20
#                     )
#                     for cell in cells:
#                         x1, y1, x2, y2 = cell
#                         draw.rectangle([x1, y1, x2, y2], outline="blue")

#                         draw_synth.rectangle([x1, y1, x2, y2], outline="blue")
#                         txt = page.get_textbox(fitz.Rect(x1, y1, x2, y2))
#                         # draw_synth.text((x1, y1), txt, fill="black")

#             page.apply_redactions()

#             for block in page.get_text("blocks"):
#                 x1, y1, x2, y2 = block[:4]
#                 content = block[4]
#                 # print(f"{content} at {x1, y1, x2, y2}")
#                 block_no = block[5]
#                 block_type = block[6] # 0: text, 1: image
#                 # draw.rectangle([x1, y1, x2, y2], outline="red")
#                 # draw_synth.text((x1, y1), content, fill="black")

#             links = page.get_links()

#             for link in links:
#                 f = link["from"]
#                 x1, y1, x2, y2 = f
#                 draw.rectangle([x1, y1, x2, y2], outline="yellow")
#                 draw_synth.rectangle([x1, y1, x2, y2], outline="yellow")

#             widgets = page.widgets()
#             for widget in widgets: # Form fields
#                 x1, y1, x2, y2 = widget.rect
#                 # draw.rectangle([x1, y1, x2, y2], outline="purple")
#                 draw_synth.rectangle([x1, y1, x2, y2], outline="purple")
#                 draw_synth.text((x2, y2), "Widget", fill="black")
#                 # How to find the bigger bounding box to tag as the whole form?


#             # drawings = page.get_drawings()
#             # for drawing in drawings:
#             #     for item in drawing["items"]:
#             #         if item[0] == "l":
#             #             x1, y1= item[1]
#             #             x2, y2 = item[2]
#             #             draw_synth.line([x1, y1, x2, y2], fill="orange")
#             #             draw_synth.text((x2, y2), "Line", fill="black")
#             #         elif item[0] == "re":
#             #             x1, y1, x2, y2 = item[1]
#             #             draw_synth.rectangle([x1, y1, x2, y2], outline="orange")
#             #             draw_synth.text((x2, y2), "Rectangle", fill="black")
#             #         elif item[0] == "qu":
#             #             x1, y1, x2, y2 = item[1]
#             #             draw_synth.rectangle([x1, y1, x2, y2], outline="orange")
#             #             draw_synth.text((x2, y2), "Square", fill="black")
#             #         elif item[0] == "c": # Bezier curve
#             #             x1, y1 = item[1]
#             #             x2, y2 = item[2]
#             #             x3, y3 = item[3]
#             #             x4, y4 = item[4]
#             #             # Bezier from (x1, y1) to (x4, y4) with control points (x2, y2) and (x3, y3)
#             #             draw_synth.line([x1, y1, x2, y2], fill="orange")
#             #             draw_synth.line([x2, y2, x3, y3], fill="orange")
#             #             draw_synth.line([x3, y3, x4, y4], fill="orange")
#             #             draw_synth.text((x4, y4), "Bezier", fill="black")


#             plot_img = Image.new("RGB", (page_img.width * 2, page_img.height))
#             plot_img.paste(page_img, (0, 0))
#             plot_img.paste(synth_img, (page_img.width, 0))
#             plot_img.save(f"{pdf_path}_{page_index}.png")

In [6]:
from collections import defaultdict
import fitz  # PyMuPDF
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from pathlib import Path


def is_empty(s):
    return not s.strip()

# External function to handle visualization using tracked annotations
def visualize_annotations(image, annotations):
    draw = ImageDraw.Draw(image)
    # Define colors for each type
    colors = {
        "table": "green",
        "table_cell": "lightgreen",  # Optional: if you want to mark cells differently
        "text": "red",
        "link": "yellow",
        "widget": "purple",
    }
    for ann in annotations:
        bbox = ann["bbox"]
        typ = ann["type"]
        if typ == "table":
            for cell in ann["cells"]:
                draw.rectangle(cell["bbox"], outline=colors["table_cell"], width=1)
        color = colors.get(typ, "black")
        draw.rectangle(bbox, outline=color, width=2)
        # Optionally, label the box with its type
        # draw.text((bbox[0], bbox[1]), f"{typ}", fill=color)
    return image

def extract_tables(page):
    res = []
    tabs = page.find_tables()
    if tabs.tables:
        for table in tabs.tables:
            table_bbox = tuple(table.bbox)
            table_obj = {
                "type": "table",
                "bbox": table_bbox,
                "cells": []
            }
            for cell in table.cells:
                cell_bbox = tuple(cell)
                cell_obj = {
                    "type": "table_cell",
                    "bbox": cell_bbox,
                    "content": page.get_textbox(fitz.Rect(*cell_bbox))
                }
                table_obj["cells"].append(cell_obj)
            res.append(table_obj)
    return res

def extract_text_blocks(page):
    res = []
    for block in page.get_text("blocks"):
        if is_empty(block[4]):
            continue
        bbox = tuple(block[:4])
        res.append(
            {
                "type": "text",
                "bbox": bbox,
                "content": block[4],
            }
        )
    return res


def extract_links(page):
    res = []
    links = page.get_links()
    for link in links:
        bbox = tuple(link["from"])
        if hasattr(link, "uri"):
            res.append({"type": "link", "bbox": bbox, "uri": link["uri"]})
    return res


def extract_widgets(page):
    "Form fields"
    res = []
    widgets = page.widgets()
    for widget in widgets:
        bbox = tuple(widget.rect)
        res.append(
            {
                "type": "widget",
                "bbox": bbox,
                "field_name": widget.field_name
            }
        )
        # How to find the bigger bounding box to tag as the whole form?
    return res

# Process a single PDF and track annotations
def process_pdf(p, num_pages=None):
    annotations = {
        "source": p,
        "pages": []
    }

    with fitz.open(p) as f:
        for page_nr, page in enumerate(f):
            page_res = {
                "page": page_nr,
                "annotations": []
            }
            table_res = extract_tables(page)
            for table in table_res:
                page.add_redact_annot(table["bbox"])
            page_res["annotations"].extend(table_res)
            page.apply_redactions()

            text_res = extract_text_blocks(page)
            page_res["annotations"].extend(text_res)

            link_res = extract_links(page)
            page_res["annotations"].extend(link_res)

            widget_res = extract_widgets(page)
            page_res["annotations"].extend(widget_res)

            annotations["pages"].append(page_res)

            if num_pages is not None and page_nr >= num_pages:
                break

    return annotations

COLORS = {
    "table": "green",
    "table_cell": "lightgreen",
    "text": "red",
    "link": "yellow",
    "widget": "purple",
}

def visualize_page(page, annotations):
    image = page.get_pixmap()
    image = Image.frombytes("RGB", [image.width, image.height], image.samples)
    draw = ImageDraw.Draw(image)
    for ann in annotations:
        bbox = ann["bbox"]
        typ = ann["type"]
        if typ == "table":
            for cell in ann["cells"]:
                draw.rectangle(cell["bbox"], outline=COLORS["table_cell"], width=1)
        color = COLORS.get(typ, "black")
        draw.rectangle(bbox, outline=color, width=2)
        draw.text((bbox[2]+3, bbox[1]), f"{typ}", fill=color)
    return image

In [9]:

# pdf_dir = r"D:\workspace\doc-understanding\data\pdf"
import traceback


pdf_dir = r"D:\library\Papers"
pdfs = [os.path.join(pdf_dir, f) for f in os.listdir(pdf_dir) if f.endswith(".pdf")]

for pdf_path in pdfs:
    # Process the PDF and track annotations
    try:
        annotations = process_pdf(pdf_path)
        for page_obj in annotations["pages"]:
            visualize_page(fitz.open(pdf_path)[page_obj["page"]], page_obj["annotations"]).save(f"{Path(pdf_path).stem}_{page_obj['page']}_annotated.png")
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")
        print(traceback.format_exc())
        break



Error processing D:\library\Papers\ddpm.pdf: list index out of range
Traceback (most recent call last):
  File "C:\Users\ihett\AppData\Local\Temp\ipykernel_20696\4068207750.py", line 11, in <module>
    annotations = process_pdf(pdf_path)
                  ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ihett\AppData\Local\Temp\ipykernel_20696\2265077897.py", line 120, in process_pdf
    link_res = extract_links(page)
               ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ihett\AppData\Local\Temp\ipykernel_20696\2265077897.py", line 74, in extract_links
    links = page.get_links()
            ^^^^^^^^^^^^^^^^
  File "d:\workspace\doc-understanding\.pdfvenv\Lib\site-packages\pymupdf\utils.py", line 1167, in get_links
    ln = ln.next
         ^^^^^^^
  File "d:\workspace\doc-understanding\.pdfvenv\Lib\site-packages\pymupdf\__init__.py", line 6560, in next
    val.xref = link_xrefs[idx + 1]
               ~~~~~~~~~~^^^^^^^^^
IndexError: list index out of range

